# Adaboost(ロジスティック回帰)

In [8]:
library(ggplot2)
library(rpart)
library(brglm)
font <- theme(text = element_text(family = "IPAGothic"))

In [9]:
df <- read.csv("Newmodel-data.csv",header=T)
dat <- df[, c(11,3,4,5,6,7,8,9,10)]
odd.number <- 2*(1:113)
odd.data <- dat[odd.number,]
even.data <- dat[-odd.number,]
dat.data <- rbind(odd.data, even.data)

In [85]:
adaboosting <- function(object, data, iter){ #一列目に応答数
    true_data <- data[,1]
    D_1 <- c()
    for (i in 1:nrow(data)){
        D_1 <- append(D_1, 1/nrow(data))
    }
    D_m <- matrix( , , nrow(data))
    D_m <- rbind(D_m, D_1)
    h_m <- list()
    Alpha_m <- c()
    ret <- list()
    for (i in 1:iter){
        dat0 <- data[,-1]
        dat <- cbind(object=data[,1], dat0*D_m[i+1,])
        h_m <- append(h_m, list(lm(dat$object~., data=dat)))
        pred <- 1/(1+exp(-predict(h_m[[i]], data, type="response")))
        pred[pred>=0.5] <- 1
        pred[pred<0.5] <- 0 
        E_m <- c()
        d_m <- c()
        for (j in 1:nrow(data)){
            if (pred[j] == true_data[j]){
                E_m <- append(E_m, 0)
                d_m <- append(d_m, D_m[i+1,j])
            } else {
                E_m <- append(E_m, D_m[i+1,j])
                d_m <- append(d_m, D_m[i+1,j])
            }
        }
        E_m <- sum(E_m)/sum(d_m)
        if (E_m > 0.5){
            break
        } else {    
            Alpha_m <- append(Alpha_m, log((1-E_m)/E_m))
            Alpha_m[is.infinite(Alpha_m)] <- 0
            d_m <- c()
            for (j in 1:nrow(data)){
                if (pred[j] == true_data[j]){
                    d_m <- append(d_m, D_m[i+1,j])
                } else {
                    d_m <- append(d_m, D_m[i+1,j]*exp(Alpha_m[i]))
                }
            }
            D_m <- rbind(D_m, d_m)
            ret <- append(ret, c(h_m[i], Alpha_m[i]))
        }
    }
    return (ret) 
}

In [86]:
even.data$natorcong <- as.numeric(as.character(even.data$natorcong))
odd.data$natorcong <- as.numeric(as.character(odd.data$natorcong))
even.data2 <- even.data[,-c(7,8)]
odd.data2 <- odd.data[,-c(7,8)]
o.ada <- adaboosting(natorcong, odd.data2, 20)
o.ada

[[1]]

Call:
lm(formula = dat$object ~ ., data = dat)

Coefficients:
(Intercept)        alpha         beta        gamma        delta      epsilon  
     0.6753      -0.4338     494.1373    -582.4986     -50.1760      16.2847  
      theta  
   -28.8491  


[[2]]
[1] 2.227078

[[3]]

Call:
lm(formula = dat$object ~ ., data = dat)

Coefficients:
(Intercept)        alpha         beta        gamma        delta      epsilon  
     0.0204      -0.1310      75.4206    -103.6535      11.5780       4.8124  
      theta  
     0.4550  


[[4]]
[1] 1.349927

[[5]]

Call:
lm(formula = dat$object ~ ., data = dat)

Coefficients:
(Intercept)        alpha         beta        gamma        delta      epsilon  
     0.1045      -0.1374     106.1914    -146.0959       7.3906       6.0826  
      theta  
    -1.4709  


[[6]]
[1] 1.791759

[[7]]

Call:
lm(formula = dat$object ~ ., data = dat)

Coefficients:
(Intercept)        alpha         beta        gamma        delta      epsilon  
    0.12833      0.06

In [87]:
obj <- matrix(, , nrow(even.data2))
for (i in 2*(1:(length(o.ada)/2))){
    obj <- rbind(obj, o.ada[[i]]*predict(o.ada[[i-1]], even.data2, type="response"))
}
logistic.ada <- 1/(1+exp(-colSums(obj[-1,])))
logistic.ada2 <- 1/(1+exp(-colSums(obj[-1,])))
logistic.ada2[logistic.ada2>=0.5] <- 1
logistic.ada2[logistic.ada2<0.5] <- 0
correct <- 100*sum(logistic.ada2==even.data2$natorcong)/nrow(even.data2)
names(correct) <- "正解率"
correct

正解率 
94.69027

In [88]:
index <- transform(logistic.ada, 残差=even.data2[,1]-logistic.ada)
index

,X_data,残差
,<dbl>,<dbl>
1,2.192131e-87,-2.192131e-87
3,1.082472e-71,-1.082472e-71
5,6.232851e-117,-6.232851e-117
7,1.688611e-116,-1.688611e-116
9,4.784300e-85,-4.784300e-85
11,2.689432e-116,-2.689432e-116
13,1.608728e-42,-1.608728e-42
15,1.715877e-10,-1.715877e-10
17,3.940467e-48,-3.940467e-48


In [89]:
summary(index)

     X_data             残差         
 Min.   :0.00000   Min.   :-1.00000  
 1st Qu.:0.00000   1st Qu.: 0.00000  
 Median :0.00000   Median : 0.00000  
 Mean   :0.09836   Mean   : 0.01669  
 3rd Qu.:0.00000   3rd Qu.: 0.00000  
 Max.   :1.00000   Max.   : 1.00000  